In [1]:
from code.data_utils import get_token, get_auth_header, get_recs
import pandas as pd

5835805e22a345948373bd954162324c 6ccb11160d65423c96d93038b719b902


In [2]:
from requests import post, get
import json 

def xlist(x):
    return (x or [])

def xtolist(x):
    #handle None type
    x = xlist(x)

    # Check if x is a string
    if isinstance(x, str):
        # Convert the string to a list containing the string
        x = [x]
    elif isinstance(x, list):
        # Check if each item in the list is a string, if not, raise an error
        if not all(isinstance(item, str) for item in x):
            raise ValueError("Each item in the input list must be a string")
    else:
        # If the input is neither a string nor a list, raise an error or handle it as needed
        raise ValueError("input-x must be a string or a list of strings")

    return x


def get_recs(token, seed_tracks=None, seed_artists=None, limit=None, max_popularity=None, 
             min_tempo=None, max_tempo=None, min_key=None, max_key=None, target_valence=None, target_key=None):
    seed_tracks = xtolist(seed_tracks)
    seed_artists = xtolist(seed_artists)

    if not seed_tracks and not seed_artists:
        raise ValueError("Must provide at least one seed track or seed artist")

    headers = get_auth_header(token)
    query_url = "https://api.spotify.com/v1/recommendations?"
    if seed_tracks:
        query_url += f"seed_tracks={'%2C'.join(seed_tracks)}"
    if seed_artists:
        query_url += f"&seed_artists={'%2C'.join(seed_artists)}"
    if isinstance(limit, int):
        query_url += f"&limit={limit}"
    if isinstance(max_popularity, int):
        query_url += f"&max_popularity={max_popularity}"
    if isinstance(min_tempo, int):
        query_url += f"&min_tempo={min_tempo}"
    if isinstance(max_tempo, int):
        query_url += f"&max_tempo={max_tempo}"
    if isinstance(min_key, int):
        query_url += f"&min_key={min_key}"
    if isinstance(max_key, int):
        query_url += f"&max_key={max_key}"
    if isinstance(target_key, int):
        query_url += f"&target_key={target_key}"
    if isinstance(target_valence, float):
        query_url += f"&target_valence={target_valence}"


    result = get(query_url, headers=headers)
    # print(result)
    js_result = json.loads(result.content)['tracks']
    # print(js_result)
    artists = [res['artists'][0]['name']  for res in js_result]
    release_dates = [res['album']['release_date'] for res in js_result]
    df_track_recs = pd.DataFrame(js_result)[['name', 'popularity']]
    df_track_recs.insert(0, 'artist', artists)
    df_track_recs.insert(0, 'release_date', release_dates)
    df_track_recs['query'] = [f"artist:\042{a}\042 track:\042{t}\042" for a,t in zip(df_track_recs['artist'].tolist(), df_track_recs['name'].tolist())]
    return df_track_recs

In [3]:
token=get_token()
print(token)

BQAEjNym3yc0usQeNVH8bJ7dXvSWoDc6hTlYQ8jPMCkUcXJFQoQ6xSsWkSYZ9wxESBXwxbzvBIrfk-1pgR7esFFiC_C8eXYhhY5LDDoPmCEEWvhfMf4


In [4]:

#Solomun - Explicit: https://open.spotify.com/track/1su8exjc2rZcEJNjuEV7Zd?si=5748ff5689e44f0f
seed_tracks = ['1su8exjc2rZcEJNjuEV7Zd']
seed_artists = None #['6u4jLGLPuarS3i2XWHVxoS']

rec_df = get_recs(token, seed_tracks=seed_tracks, seed_artists=seed_artists, limit=10, max_popularity=None)
with pd.option_context('display.max_colwidth', None): 
    # display(rec_df.sort_values('release_date', ascending=True))
    display(rec_df)

,release_date,artist,name,popularity,query
0,2014-05-05,Claude VonStroke,The Clapping Track - Solomun Remix,19,"artist:""Claude VonStroke"" track:""The Clapping Track - Solomun Remix"""
1,2010-05-10,DJ Kaos,Hard to Earn - Solomun Remix,2,"artist:""DJ Kaos"" track:""Hard to Earn - Solomun Remix"""
2,2016-04-29,Booka Shade,Neon Gods - Rodriguez Jr. Remix,13,"artist:""Booka Shade"" track:""Neon Gods - Rodriguez Jr. Remix"""
3,2014-02-21,Boris Brejcha,Puki,25,"artist:""Boris Brejcha"" track:""Puki"""
4,2022-06-24,Christian Löffler,Solo - Solomun Remix,20,"artist:""Christian Löffler"" track:""Solo - Solomun Remix"""
5,2017-11-06,Solomun,Amanacer,30,"artist:""Solomun"" track:""Amanacer"""
6,2016-07-01,Lana Del Rey,Cornflake Boy - Solomun Dub Remix,4,"artist:""Lana Del Rey"" track:""Cornflake Boy - Solomun Dub Remix"""
7,2018-01-19,Super Flu,Mygut - Solomun Remix Radio Edit,12,"artist:""Super Flu"" track:""Mygut - Solomun Remix Radio Edit"""
8,2013-04-24,Tale Of Us,Discochord,26,"artist:""Tale Of Us"" track:""Discochord"""
9,2013-01-31,Foals,Late Night - Solomun Remix,36,"artist:""Foals"" track:""Late Night - Solomun Remix"""
